In [1]:
import tensorflow as tf
import numpy as np

In [2]:
idx2char = ['h','i','e','l','o']
# hihello 문자열, hihell -> ihello
xdata = [[0,1,0,2,3,3]] #hihell (입력)
# [None,sequence_length,input_dim] 
xOneHot = [[
    [1,0,0,0,0], #h
    [0,1,0,0,0], #i
    [1,0,0,0,0], #h
    [0,0,1,0,0], #e
    [0,0,0,1,0], #l
    [0,0,0,1,0]  #l
]]
ydata = [[1,0,2,3,3,4]] #ihello (출력)

In [3]:
np.shape(xOneHot)

(1, 6, 5)

In [4]:
num_classes=5
input_dim=5  #원핫인코딩 했을 때의 크기
hidden_size=5  #RNN(LSTM) CELL에서 출력의 크기
batch_size=1  #학습하고자 하는 문장 1개
sequence_length=6 #출력되는 문자의 길이
learning_rate=0.1

In [5]:
x = tf.placeholder(tf.float32, [None,sequence_length,input_dim])
y = tf.placeholder(tf.int32, [None,sequence_length])

In [6]:
cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
output, _states = tf.nn.dynamic_rnn(cell, x, initial_state=initial_state, dtype=tf.float32)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [7]:
# Ct-1 : 이전 주어의 성별 정보 * ft

In [8]:
xForFc = tf.reshape(output,[-1,hidden_size])
output = tf.contrib.layers.fully_connected(inputs=xForFc, num_outputs=num_classes, activation_fn=None)
output = tf.reshape(output, [batch_size, sequence_length, num_classes])

Instructions for updating:
Please use `layer.__call__` method instead.


In [9]:
weights = tf.ones([batch_size, sequence_length])

In [10]:
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=output, targets=y, weights=weights)

In [11]:
loss =  tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [12]:
prediction = tf.argmax(output, axis=2)

In [13]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(100):
        lossv, _ = sess.run([loss,train], feed_dict={x:xOneHot, y:ydata})
        res = sess.run(prediction, feed_dict={x:xOneHot})
        print(i, "cost:", lossv, "prediction:", res, "true Y:", ydata)
        
        s = [idx2char[c] for c in np.squeeze(res)]
        print("\nprediction str:", "".join(s))          

0 cost: 1.6656989 prediction: [[2 3 3 2 3 3]] true Y: [[1, 0, 2, 3, 3, 4]]

prediction str: ellell
1 cost: 1.5590636 prediction: [[3 3 3 3 3 3]] true Y: [[1, 0, 2, 3, 3, 4]]

prediction str: llllll
2 cost: 1.5031433 prediction: [[3 3 3 3 3 3]] true Y: [[1, 0, 2, 3, 3, 4]]

prediction str: llllll
3 cost: 1.4413904 prediction: [[3 3 3 3 3 3]] true Y: [[1, 0, 2, 3, 3, 4]]

prediction str: llllll
4 cost: 1.3661939 prediction: [[3 3 3 3 3 3]] true Y: [[1, 0, 2, 3, 3, 4]]

prediction str: llllll
5 cost: 1.2736318 prediction: [[3 3 3 3 3 4]] true Y: [[1, 0, 2, 3, 3, 4]]

prediction str: lllllo
6 cost: 1.1587087 prediction: [[3 3 3 3 3 4]] true Y: [[1, 0, 2, 3, 3, 4]]

prediction str: lllllo
7 cost: 1.0231987 prediction: [[3 0 3 3 3 4]] true Y: [[1, 0, 2, 3, 3, 4]]

prediction str: lhlllo
8 cost: 0.8769531 prediction: [[3 0 3 3 3 4]] true Y: [[1, 0, 2, 3, 3, 4]]

prediction str: lhlllo
9 cost: 0.7395393 prediction: [[3 0 2 3 3 4]] true Y: [[1, 0, 2, 3, 3, 4]]

prediction str: lhello
10 cost: 0

In [14]:
# [1,2]  =>  axis=0  => [[1,2]]
# [1,2]  =>  axis=1  => [[1],[2]]
   
x = np.array([1,2])
x.shape
y = np.expand_dims(x, axis=1)  #차원 늘리기
y.shape

(2, 1)

In [15]:
y

array([[1],
       [2]])

In [16]:
x = np.array([[[0],[1],[2]]])
x.shape

(1, 3, 1)

In [17]:
np.squeeze(x)

array([0, 1, 2])

In [18]:
tf.reset_default_graph()

In [19]:
hidden_size=2
cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size)
# 출력 shape = (1,1,2)

In [20]:
data = np.array([[[1,0,0,0]]], dtype=np.float32)
# xdata shape = (1,1,4)

In [21]:
sess = tf.Session()
output, _states = tf.nn.dynamic_rnn(cell, xdata, dtype=tf.float32)
sess.run(tf.global_variables_initializer())
print(sess.run(output))

ValueError: Expected input tensor Tensor("rnn/Const:0", shape=(), dtype=int32) to have rank at least 2

In [ ]:
# input shape = (단어 개수, 입력글자수, 원핫 크기)
# output shape = (단어 개수, 입력글자수)

In [22]:
tf.reset_default_graph()

In [ ]:
# h -> [1,0,0,0]
xdata = np.array([[h,e,l,l,o],
                  [e,o,l,l,l],
                  [l,l,e,e,l]], dtype=np.float32)
hidden_size=2
cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size) #출력: 2개
sess = tf.Session()
output, _states = tf.nn.dynamic_rnn(cell, xdata, dtype=tf.float32)
sess.run(tf.global_variables_initializer())
print(sess.run(output))

In [25]:
from tensorflow.examples.tutorials.mnist import input_data

In [26]:
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/datas

In [27]:
lr=0.01
total_epoch=30
batch_size=128

n_input=28 #한번에 입력받는 데이터 개수
n_step=28 #28줄
n_hidden=128 #셀에서 나가는 출력의 개수
n_class=10  #숫자의 종류(0~9)

In [28]:
#신경망
x = tf.placeholder(tf.float32, [None,n_step,n_input])
y = tf.placeholder(tf.float32, [None,n_class])

In [32]:
w = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

In [29]:
#rnn 학습할 셀을 생성
cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)

In [31]:
#rnn 망 구성
outputs, _states = tf.nn.dynamic_rnn(cell, x, dtype=tf.float32)
#outputs : [batch_size, n_step, n_hidden]
# => 구조 변경(y형태로) => y:[batch_size, n_class]

#      0    ,  1    ,   2
#[batch_size, n_step, n_hidden] -> 
# [n_step, batch_size, n_hidden]
#     1    ,  0      ,  2
outputs = tf.transpose(outputs, [1,0,2])
outputs = outputs[-1]

In [33]:
model = tf.matmul(outputs,w) + b
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=y))
opt = tf.train.AdamOptimizer(lr).minimize(cost)

In [34]:
#신경망 모델 학습
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [35]:
total_batch = int(mnist.train.num_examples/batch_size)

In [36]:
for epoch in range(total_epoch):
    total_cost=0
    for i in range(total_batch):
        batchX, batchY = mnist.train.next_batch(batch_size)
        batchX = batchX.reshape((batch_size, n_step, n_input))
        #128,784 -> 128,28,28
        _, costv = sess.run([opt, cost], feed_dict={x:batchX, y:batchY})
        total_cost+=costv
    print("에폭:", "%4d" %(epoch+1), "cost 평균:", "{:.3f}".format(total_cost/total_batch))

에폭:    1 cost 평균: 0.344
에폭:    2 cost 평균: 0.088
에폭:    3 cost 평균: 0.063
에폭:    4 cost 평균: 0.048
에폭:    5 cost 평균: 0.043
에폭:    6 cost 평균: 0.045
에폭:    7 cost 평균: 0.034
에폭:    8 cost 평균: 0.035
에폭:    9 cost 평균: 0.035
에폭:   10 cost 평균: 0.029
에폭:   11 cost 평균: 0.030
에폭:   12 cost 평균: 0.028
에폭:   13 cost 평균: 0.026
에폭:   14 cost 평균: 0.029
에폭:   15 cost 평균: 0.025
에폭:   16 cost 평균: 0.025
에폭:   17 cost 평균: 0.022
에폭:   18 cost 평균: 0.022
에폭:   19 cost 평균: 0.025
에폭:   20 cost 평균: 0.025
에폭:   21 cost 평균: 0.019
에폭:   22 cost 평균: 0.022
에폭:   23 cost 평균: 0.023
에폭:   24 cost 평균: 0.021
에폭:   25 cost 평균: 0.020
에폭:   26 cost 평균: 0.021
에폭:   27 cost 평균: 0.020
에폭:   28 cost 평균: 0.017
에폭:   29 cost 평균: 0.020
에폭:   30 cost 평균: 0.016


In [37]:
isCorrect = tf.equal(tf.argmax(model,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(isCorrect, tf.float32))
testSize = len(mnist.test.images)
testX = mnist.test.images.reshape(testSize, n_step, n_input)
testY = mnist.test.labels

print(sess.run(accuracy, feed_dict={x:testX, y:testY}))

0.9853
